# **Question 1**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import USPS
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import precision_recall_curve, auc

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Load USPS dataset
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = USPS(root='./data', train=True, download=True, transform=transform)
test_dataset = USPS(root='./data', train=False, download=True, transform=transform)

batch_size = 64

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


input_size = 28*28
hidden_size = 128
num_classes = 10
learning_rate = 0.001
num_epochs = 10


mlp_model = MLP(input_size, hidden_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
mlp_optimizer = optim.Adam(mlp_model.parameters(), lr=learning_rate)

# TensorBoard logging
writer = SummaryWriter()


for epoch in range(num_epochs):
    for images, labels in train_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)


        outputs = mlp_model(images)
        loss = criterion(outputs, labels)


        mlp_optimizer.zero_grad()
        loss.backward()
        mlp_optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


mlp_model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = mlp_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        precision, recall, _ = precision_recall_curve(all_labels, all_preds, pos_label=1)
        pr_auc = auc(recall, precision)

        # Log precision-recall curve
        writer.add_pr_curve('Precision-Recall Curve', torch.tensor(all_labels), torch.tensor(all_preds), epoch)

        # Log PR AUC
        writer.add_scalar('PR AUC', pr_auc, epoch)
mlp_accuracy = correct / total
mlp_precision = precision_score(all_labels, all_preds, average='macro')
mlp_recall = recall_score(all_labels, all_preds, average='macro')
mlp_confusion_matrix = confusion_matrix(all_labels, all_preds)

print('MLP Accuracy: {:.4f}'.format(mlp_accuracy))
print('MLP Precision: {:.4f}'.format(mlp_precision))
print('MLP Recall: {:.4f}'.format(mlp_recall))
print('MLP Confusion Matrix:')
print(mlp_confusion_matrix)

writer.close()

Epoch [1/10], Loss: 0.2628
Epoch [2/10], Loss: 0.1188
Epoch [3/10], Loss: 0.2470
Epoch [4/10], Loss: 0.1764
Epoch [5/10], Loss: 0.1250
Epoch [6/10], Loss: 0.0496
Epoch [7/10], Loss: 0.0831
Epoch [8/10], Loss: 0.0356
Epoch [9/10], Loss: 0.0791
Epoch [10/10], Loss: 0.0723
MLP Accuracy: 0.9402
MLP Precision: 0.9385
MLP Recall: 0.9329
MLP Confusion Matrix:
[[354   0   2   0   1   0   1   0   1   0]
 [  0 253   1   1   3   0   4   0   1   1]
 [  2   0 188   2   1   2   1   0   2   0]
 [  3   0   3 142   0  12   0   0   5   1]
 [  0   2   4   0 187   1   1   0   0   5]
 [  3   0   0   4   2 149   0   0   0   2]
 [  1   0   3   0   2   0 164   0   0   0]
 [  1   0   5   1   6   0   0 130   2   2]
 [  5   0   3   2   1   2   0   0 150   3]
 [  0   1   1   0   2   2   0   0   1 170]]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import USPS
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from torch.utils.tensorboard import SummaryWriter


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 4 * 4)  # Flatten the feature maps
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = USPS(root='./data', train=True, download=True, transform=transform)
test_dataset = USPS(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)


cnn_model = CNN()

criterion = nn.CrossEntropyLoss()
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)


writer = SummaryWriter()


cnn_model.train()
for epoch in range(5):  # Train for 5 epochs
    for images, labels in train_loader:
        cnn_optimizer.zero_grad()
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        cnn_optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

cnn_model.eval()
cnn_predictions = []
cnn_true_labels = []
for images, labels in test_loader:
    outputs = cnn_model(images)
    _, predicted = torch.max(outputs.data, 1)
    cnn_predictions.extend(predicted.numpy())
    cnn_true_labels.extend(labels.numpy())

cnn_accuracy = accuracy_score(cnn_true_labels, cnn_predictions)
print("CNN Accuracy:", cnn_accuracy)

cnn_precision = precision_score(cnn_true_labels, cnn_predictions, average='macro')
cnn_recall = recall_score(cnn_true_labels, cnn_predictions, average='macro')
cnn_confusion_matrix = confusion_matrix(cnn_true_labels, cnn_predictions)

print('CNN Accuracy: {:.4f}'.format(cnn_accuracy))
print('CNN Precision: {:.4f}'.format(cnn_precision))
print('CNN Recall: {:.4f}'.format(cnn_recall))
print('CNN Confusion Matrix:')
print(cnn_confusion_matrix)

writer.close()

Epoch [1/10], Loss: 0.2256
Epoch [2/10], Loss: 0.1768
Epoch [3/10], Loss: 0.1029
Epoch [4/10], Loss: 0.1881
Epoch [5/10], Loss: 0.0897
CNN Accuracy: 0.9412057797708022
CNN Accuracy: 0.9412
CNN Precision: 0.9373
CNN Recall: 0.9367
CNN Confusion Matrix:
[[347   0   1   0   2   3   4   1   0   1]
 [  0 256   0   0   3   0   3   2   0   0]
 [  2   0 185   5   2   2   0   2   0   0]
 [  0   0   3 146   0  16   0   0   0   1]
 [  0   2   2   0 188   0   2   2   0   4]
 [  1   0   0   3   0 154   0   0   0   2]
 [  0   0   2   0   3   2 163   0   0   0]
 [  0   0   1   1   4   0   0 140   0   1]
 [  5   1   1   4   1   9   0   1 139   5]
 [  0   1   0   0   1   1   0   3   0 171]]


# **Question 2**

In [ ]:
import torchvision.datasets
import torch.utils.data as data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageNet


model = models.resnet101(pretrained=True)


num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)


transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

from torchvision.datasets.imagenet import parse_devkit_archive, parse_train_archive, parse_val_archive


train_dataset = ImageNet(root='/content/drive/MyDrive/Dlops', transform=transform, train=True)
test_dataset = ImageNet(root='/content/drive/MyDrive/Dlops', transform=transform, train=False)

batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


criterion = nn.CrossEntropyLoss()


optimizers = [
    optim.Adam(model.parameters()),
    optim.Adagrad(model.parameters()),
    optim.RMSprop(model.parameters())
]


num_epochs = 10
for optimizer in optimizers:
    optimizer_name = optimizer.__class__.__name__
    print(f"Training with {optimizer_name}")

    train_losses = []
    train_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = correct / total

        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_accuracy)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")




RuntimeError: The archive ILSVRC2012_img_train.tar is not present in the root directory or is corrupted. You need to download it externally and place it in /content/drive/MyDrive/Dlops.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import USPS
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from torch.utils.tensorboard import SummaryWriter

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 4 * 4)  # Flatten the feature maps
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = USPS(root='./data', train=True, download=True, transform=transform)
test_dataset = USPS(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

cnn_model = CNN()

criterion = nn.CrossEntropyLoss()
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

writer = SummaryWriter()

cnn_model.train()
num_epochs = 5
for epoch in range(num_epochs):  # Train for 5 epochs
    for images, labels in train_loader:
        cnn_optimizer.zero_grad()
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        cnn_optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the trained model
torch.save(cnn_model.state_dict(), 'cnn_model.pth')
print("CNN model saved successfully")

cnn_model.eval()
cnn_predictions = []
cnn_true_labels = []
for images, labels in test_loader:
    outputs = cnn_model(images)
    _, predicted = torch.max(outputs.data, 1)
    cnn_predictions.extend(predicted.numpy())
    cnn_true_labels.extend(labels.numpy())

cnn_accuracy = accuracy_score(cnn_true_labels, cnn_predictions)
print("CNN Accuracy:", cnn_accuracy)

cnn_precision = precision_score(cnn_true_labels, cnn_predictions, average='macro')
cnn_recall = recall_score(cnn_true_labels, cnn_predictions, average='macro')
cnn_confusion_matrix = confusion_matrix(cnn_true_labels, cnn_predictions)

print('CNN Accuracy: {:.4f}'.format(cnn_accuracy))
print('CNN Precision: {:.4f}'.format(cnn_precision))
print('CNN Recall: {:.4f}'.format(cnn_recall))
print('CNN Confusion Matrix:')
print(cnn_confusion_matrix)

writer.close()


100%|██████████| 6579383/6579383 [00:01<00:00, 4545258.50it/s]


100%|██████████| 1831726/1831726 [00:01<00:00, 1619517.41it/s]


Epoch [1/5], Loss: 0.4830
Epoch [2/5], Loss: 0.1224
Epoch [3/5], Loss: 0.1047
Epoch [4/5], Loss: 0.2303
Epoch [5/5], Loss: 0.0245
CNN model saved successfully
CNN Accuracy: 0.931738913801694
CNN Accuracy: 0.9317
CNN Precision: 0.9268
CNN Recall: 0.9271
CNN Confusion Matrix:
[[350   0   1   1   2   0   4   0   0   1]
 [  0 252   1   1   1   0   6   2   0   1]
 [  1   0 186   5   0   1   0   1   3   1]
 [  0   0   3 156   0   6   0   0   0   1]
 [  0   1   3   0 163   3   2   0   2  26]
 [  1   0   0  10   0 147   0   0   0   2]
 [  0   0   2   0   2   4 160   0   2   0]
 [  0   0   1   3   3   0   0 135   0   5]
 [  4   0   0   7   0   2   0   2 148   3]
 [  0   0   0   1   0   0   0   2   1 173]]


In [1]:
ls


drive/  sample_data/
